
Top words used across the document

In [0]:
%sql

select word_new, count(*) as freq from topwords group by word_new order by freq desc

In [0]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol="word_new", outputCol="rawFeatures", vocabSize=10000, minDF=5)
cvmodel = cv.fit(news_df)
featurizedData = cvmodel.transform(news_df)

In [0]:
display(featurizedData)

In [0]:
vocab = cvmodel.vocabulary
vocab_broadcast = sc.broadcast(vocab)
vocab

In [0]:
from pyspark.ml.feature import IDF

idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

In [0]:
display(rescaledData)

In [0]:
rescaledData.printSchema()

In [0]:
corpus = rescaledData.select('id', 'features').cache()
display(corpus)

In [0]:
from pyspark.ml.clustering import LDA

lda = LDA(k=20, maxIter=50, optimizer="em")
model = lda.fit(corpus)

In [0]:
ll = model.logLikelihood(corpus)
lp = model.logPerplexity(corpus)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

In [0]:
topicwords=20
topics = model.describeTopics(topicwords)
print("The topics described by their top-weighted terms:")
topics.show(truncate=False)

In [0]:
tRDD = topics.rdd.map(list)

In [0]:
topics.printSchema()

In [0]:
tRDD.collect()

In [0]:
def topic_vocab(topic):
    print(topic)
    topicNum = topic[0]
    terms = topic[1]
    weight = topic[2]
    result = []
    for i in range(topicwords):
        term = vocab[terms[i]]
        out = str(topicNum)+","+term+","+str(weight[i])
        result.append(out)
    return result

In [0]:
tRDD.map(lambda topic: topic_vocab(topic)).collect()

In [0]:
topic = tRDD.map(lambda topic: topic_vocab(topic))

In [0]:
ng_pd = spark.createDataFrame(topic).toPandas()

In [0]:
ng_pd.T